# 2 start pods on Azure AKS

change ${PJ_ROOT} to your directory.

In [ ]:
export PJ_ROOT="${HOME}/container-centric-fiware-demonstration"
cd ${PJ_ROOT};pwd

example)
```
/Users/user/container-centric-fiware-demonstration
```

## load environment variables

In [ ]:
source ${PJ_ROOT}/docs/azure_aks/env

## login AKS

In [ ]:
az login --tenant ${TENANT}

## start etcd cluster on AKS
[etcd](https://github.com/coreos/etcd)

In [ ]:
helm install stable/etcd-operator --name ccfd

In [ ]:
POD=$(kubectl get pods -l app=ccfd-etcd-operator-etcd-operator --namespace default --output name)
kubectl logs $POD --namespace=default

expected)
```
time="2018-07-11T22:52:48Z" level=info msg="Event(v1.ObjectReference{Kind:\"Endpoints\", Namespace:\"default\", Name:\"etcd-operator\", UID:\"21e458f5-855d-11e8-a3b4-026b849f8c40\", APIVersion:\"v1\", ResourceVersion:\"1580\", FieldPath:\"\"}): type: 'Normal' reason: 'LeaderElection' ccfd-etcd-operator-etcd-operator-6d5f994f86-2z65g became leader"
```

In [ ]:
kubectl apply -f etcd/etcd-cluster.yaml

In [ ]:
kubectl get pods -l app=etcd

example)
```
NAME                      READY     STATUS    RESTARTS   AGE
etcd-cluster-m5mk57c68n   0/1       Running   0          6s
etcd-cluster-nl872jt7zs   1/1       Running   0          46s
etcd-cluster-xds88qqcjx   1/1       Running   0          22s
```

In [ ]:
kubectl get services -l app=etcd

example)
```
NAME                  TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)             AGE
etcd-cluster          ClusterIP   None         <none>        2379/TCP,2380/TCP   1m
etcd-cluster-client   ClusterIP   10.0.68.3    <none>        2379/TCP            1m
```

In [ ]:
kubectl run --rm -it etcdclient --image quay.io/coreos/etcd --restart=Never -- etcdctl --peers http://etcd-cluster-client:2379 member list

example)
```
477d8981f585fdf3: name=etcd-cluster-xds88qqcjx peerURLs=http://etcd-cluster-xds88qqcjx.etcd-cluster.default.svc:2380 clientURLs=http://etcd-cluster-xds88qqcjx.etcd-cluster.default.svc:2379 isLeader=false
6fc6ea1e06f2b405: name=etcd-cluster-nl872jt7zs peerURLs=http://etcd-cluster-nl872jt7zs.etcd-cluster.default.svc:2380 clientURLs=http://etcd-cluster-nl872jt7zs.etcd-cluster.default.svc:2379 isLeader=true
8b8708e453bef839: name=etcd-cluster-m5mk57c68n peerURLs=http://etcd-cluster-m5mk57c68n.etcd-cluster.default.svc:2380 clientURLs=http://etcd-cluster-m5mk57c68n.etcd-cluster.default.svc:2379 isLeader=false
```

## start vernemq cluster on AKS
[vernemq](https://vernemq.com/)

### create usernames & passwords of vernemq

In [ ]:
mkdir -p secrets
rm -f secrets/vmq.passwd
touch secrets/vmq.passwd

In [ ]:
for e in $(env); do
  if [[ "${e}" =~ ^VERNEMQ__([[:alnum:]_-]+)=([[:alnum:]_-]+)$ ]]; then
    username=${BASH_REMATCH[1]}
    password=${BASH_REMATCH[2]}
    
    expect -c "
    spawn docker run --rm -v $(pwd)/secrets:/mnt -it erlio/docker-vernemq vmq-passwd /mnt/vmq.passwd ${username}
    expect \"Password:\"
    send -- \"${password}\n\"
    expect \"Reenter password:\"
    send -- \"${password}\n\"
    expect \"\\\$\"
    exit 0
    "
  fi
done

### register vmq.passwd to kubernetes secret

In [ ]:
kubectl create secret generic vernemq-passwd --from-file=./secrets/vmq.passwd

### create tls cert files using let's encrypt

_Outside of this notebook_
1. open a ternminal.
1. run a command displayed below.

In [ ]:
echo "docker run -it -v ${PJ_ROOT}/secrets:/etc/letsencrypt certbot/certbot certonly --manual --domain mqtt.${DOMAIN} --email ${EMAIL} --no-eff-email --agree-tos --manual-public-ip-logging-ok --preferred-challenges dns"

_Outside of this notebook_
1. wait until to add a DNS TXT Record like below.

notice) **replace XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX with the 'DNS TXT record' value displayed in opened terminal.**

In [ ]:
az network dns record-set txt add-record --resource-group ${DNS_ZONE_RG} --zone-name "${DOMAIN}" --record-set-name "_acme-challenge.mqtt" --value "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"

_Outside of this notebook_
1. in order to continue creating cert file process, press enter key in the opened terminal.
1. confirm to be displayed the below message at the opened terminal.
```
Congratulations! Your certificate and chain have been saved at:
```

notice) **replace XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX with the 'DNS TXT record' value displayed in opened terminal.**

In [ ]:
az network dns record-set txt remove-record --resource-group ${DNS_ZONE_RG} --zone-name "${DOMAIN}" --record-set-name "_acme-challenge.mqtt" --value "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"

In [ ]:
cat secrets/DST_Root_CA_X3.pem secrets/archive/mqtt.${DOMAIN}/chain1.pem > secrets/ca.crt
cp secrets/archive/mqtt.${DOMAIN}/fullchain1.pem secrets/server.crt
cp secrets/archive/mqtt.${DOMAIN}/privkey1.pem secrets/server.key

### register cert files of vernemq to kubernetes secrets

In [ ]:
kubectl create secret generic vernemq-certifications --from-file=./secrets/ca.crt --from-file=./secrets/server.crt --from-file=./secrets/server.key

### start vernemq

In [ ]:
kubectl apply -f vernemq/vernemq-cluster-azure.yaml

In [ ]:
kubectl get pods -l app=vernemq

example)
```
NAME        READY     STATUS    RESTARTS   AGE
vernemq-0   1/1       Running   0          1m
vernemq-1   1/1       Running   0          1m
vernemq-2   1/1       Running   0          33s
```

In [ ]:
kubectl exec vernemq-0 -- vmq-admin cluster show

expected)
```
+---------------------------------------------------+-------+
|                       Node                        |Running|
+---------------------------------------------------+-------+
|VerneMQ@vernemq-0.vernemq.default.svc.cluster.local| true  |
|VerneMQ@vernemq-1.vernemq.default.svc.cluster.local| true  |
|VerneMQ@vernemq-2.vernemq.default.svc.cluster.local| true  |
+---------------------------------------------------+-------+
```

**confirm that "EXTERNAL-IP" is assigned**

In [ ]:
kubectl get services -l app=mqtts

expected)
```
NAME      TYPE           CLUSTER-IP   EXTERNAL-IP   PORT(S)          AGE
mqtts     LoadBalancer   10.0.43.26   XX.XX.XX.XX   8883:32301/TCP   5m
```

### register DNS A Record for VerneMQ

In [ ]:
MQTTS_IPADDR=$(kubectl get services -l app=mqtts -o json | jq '.items[0].status.loadBalancer.ingress[0].ip' -r)
az network dns record-set a add-record --resource-group ${DNS_ZONE_RG} --zone-name "${DOMAIN}" --record-set-name "mqtt" --ipv4-address "${MQTTS_IPADDR}"

In [ ]:
nslookup mqtt.${DOMAIN}

In [ ]:
mosquitto_pub -h mqtt.${DOMAIN} -p 8883 --cafile ${PJ_ROOT}/secrets/ca.crt -d -u iotagent -P ${VERNEMQ__iotagent} -t /test -m "test"

example)
```
Client mosqpub|52150-Nobuyukin sending CONNECT
Client mosqpub|52150-Nobuyukin received CONNACK
Client mosqpub|52150-Nobuyukin sending PUBLISH (d0, q0, r0, m1, '/test', ... (4 bytes))
Client mosqpub|52150-Nobuyukin sending DISCONNECT
```

## start mondodb cluster on AKS
[mongodb](https://www.mongodb.com/)

create 'mongodb' takes a few minutes.

In [ ]:
kubectl apply -f mongodb/mongodb-cluster-azure.yaml

In [ ]:
kubectl get PersistentVolumeClaims -l app=mongodb

example)
```
NAME                              STATUS    VOLUME                                     CAPACITY   ACCESS MODES   STORAGECLASS      AGE
mongodb-storage-claim-mongodb-0   Bound     pvc-386ac3ba-856e-11e8-a3b4-026b849f8c40   30Gi       RWO            managed-premium   5m
mongodb-storage-claim-mongodb-1   Bound     pvc-88455199-856e-11e8-a3b4-026b849f8c40   30Gi       RWO            managed-premium   3m
mongodb-storage-claim-mongodb-2   Bound     pvc-b74ec32c-856e-11e8-a3b4-026b849f8c40   30Gi       RWO            managed-premium   2m
```

In [ ]:
kubectl get pods -l app=mongodb

example)
```
NAME        READY     STATUS    RESTARTS   AGE
mongodb-0   2/2       Running   0          6m
mongodb-1   2/2       Running   0          4m
mongodb-2   2/2       Running   0          2m
```

In [ ]:
kubectl get services -l app=mongodb

example)
```
NAME      TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)     AGE
mongodb   ClusterIP   None         <none>        27017/TCP   6m
```

In [ ]:
kubectl exec mongodb-0 -c mongodb -- mongo --eval 'printjson(rs.status().members.map(function(e) {return {name: e.name, stateStr:e.stateStr};}))'

expected)
```
MongoDB shell version v3.6.6
connecting to: mongodb://127.0.0.1:27017
MongoDB server version: 3.6.6
[
	{
		"name" : "mongodb-0.mongodb.default.svc.cluster.local:27017",
		"stateStr" : "PRIMARY"
	},
	{
		"name" : "mongodb-1.mongodb.default.svc.cluster.local:27017",
		"stateStr" : "SECONDARY"
	},
	{
		"name" : "mongodb-2.mongodb.default.svc.cluster.local:27017",
		"stateStr" : "SECONDARY"
	}
]
```

## start ambassador on AKS

[ambassador](https://www.getambassador.io/)

### create tls cert files using let's encrypt

_Outside of this notebook_
1. open a ternminal.
1. run a command displayed below.

In [ ]:
echo "docker run -it -v ${PJ_ROOT}/secrets:/etc/letsencrypt certbot/certbot certonly --manual --domain api.${DOMAIN} --email ${EMAIL} --no-eff-email --agree-tos --manual-public-ip-logging-ok --preferred-challenges dns"

_Outside of this notebook_
1. wait until to add a DNS TXT Record like below.

notice) **replace YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY with the 'DNS TXT record' value displayed in opened terminal.**

In [ ]:
az network dns record-set txt add-record --resource-group ${DNS_ZONE_RG} --zone-name "${DOMAIN}" --record-set-name "_acme-challenge.api" --value "YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY"

_Outside of this notebook_
1. in order to continue creating cert file process, press enter key in the opened terminal.
1. confirm to be displayed the below message at the opened terminal.
```
Congratulations! Your certificate and chain have been saved at:
```

notice) **replace YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY with the 'DNS TXT record' value displayed in opened terminal.**

In [ ]:
az network dns record-set txt remove-record --resource-group ${DNS_ZONE_RG} --zone-name "${DOMAIN}" --record-set-name "_acme-challenge.api" --value "YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY"

### register cert files of ambassador to kubernetes secrets

In [ ]:
kubectl create secret tls ambassador-certs --cert=${PJ_ROOT}/secrets/live/api.${DOMAIN}/fullchain.pem --key=${PJ_ROOT}/secrets/live/api.${DOMAIN}/privkey.pem

### start ambassador

In [ ]:
kubectl apply -f ambassador/ambassador-azure.yaml

In [ ]:
kubectl get pods -l service=ambassador

example)
```
NAME                         READY     STATUS    RESTARTS   AGE
ambassador-cf99ff6bb-45jpl   1/1       Running   0          39s
ambassador-cf99ff6bb-8kht7   1/1       Running   0          39s
ambassador-cf99ff6bb-lv2tk   1/1       Running   0          39s
```

**confirm that "EXTERNAL-IP" is assigned**

In [ ]:
kubectl get services -l service=ambassador

example)
```
NAME         TYPE           CLUSTER-IP    EXTERNAL-IP   PORT(S)                      AGE
ambassador   LoadBalancer   10.0.191.59   YY.YY.YY.YY   443:30357/TCP,80:32755/TCP   4m
```

### register DNS A Record for ambassador

In [ ]:
HTTPS_IPADDR=$(kubectl get services -l service=ambassador -o json | jq '.items[0].status.loadBalancer.ingress[0].ip' -r)
az network dns record-set a add-record --resource-group ${DNS_ZONE_RG} --zone-name "${DOMAIN}" --record-set-name "api" --ipv4-address "${HTTPS_IPADDR}"

In [ ]:
nslookup api.${DOMAIN}

In [ ]:
curl -i https://api.${DOMAIN}

expected)
```
HTTP/1.1 404 Not Found
date: Fri, 25 May 2018 00:47:41 GMT
server: envoy
content-length: 0
```

## start authorization & authentication service on AKS

### create `secrets/auth-tokens.json`
example)
```json
{
  "bearer_tokens": [
      {
          "token": "iRGTsKKHwgjf4rR2XMSN3oE9Dhm6ym3O",
          "allowed_paths": ["^/orion/.*$", "^/idas/.*$"]
      }, {
          "token": "4Xc1GFa2D8zkZRbkdygm902oGYeUAJno",
          "allowed_paths": ["^/idas/.*$"]
      }
  ],
  "basic_auths": [
      {
          "username": "user1",
          "password": "P@ssw0rd",
          "allowed_paths": ["/controller/web/"]
      }, {
          "username": "user2",
          "password": "P@ssw0rd",
          "allowed_paths": ["/controller/web/"]
      }
  ]
}
```

In [ ]:
cat << __EOS__ > secrets/auth-tokens.json
{
  "bearer_tokens": [
      {
          "token": "$(cat /dev/urandom | LC_CTYPE=C tr -dc 'a-zA-Z0-9' | head -c 32)",
          "allowed_paths": ["^/orion/.*$", "^/idas/.*$"]
      }
  ],
  "basic_auths": [
      {
          "username": "user1",
          "password": "$(cat /dev/urandom | LC_CTYPE=C tr -dc 'a-zA-Z0-9' | head -c 16)",
          "allowed_paths": ["/controller/web/"]
      }
  ]
}
__EOS__

### register auth-tokens to kubernetes secrets

In [ ]:
kubectl create secret generic auth-tokens --from-file=./secrets/auth-tokens.json

### start fiware-ambassador-auth

In [ ]:
kubectl apply -f ambassador/fiware-ambassador-auth.yaml

In [ ]:
kubectl get pods -l pod=ambassador-auth

example)
```
NAME                           READY     STATUS    RESTARTS   AGE
ambassador-auth-6fffdbd9c9-7kkpr   1/1       Running   0          56s
ambassador-auth-6fffdbd9c9-qxw6m   1/1       Running   0          56s
ambassador-auth-6fffdbd9c9-sdn5b   1/1       Running   0          56s
```

In [ ]:
kubectl get services -l service=ambassador-auth

example)
```
NAME          TYPE        CLUSTER-IP     EXTERNAL-IP   PORT(S)    AGE
ambassador-auth   ClusterIP   10.0.129.102   <none>        3000/TCP   2m
```

In [ ]:
curl -i https://api.${DOMAIN}/

example)
```
HTTP/1.1 401 Unauthorized
content-type: application/json; charset=utf-8
www-authenticate: Bearer realm="token_required"
date: Thu, 12 Jul 2018 02:21:24 GMT
content-length: 60
x-envoy-upstream-service-time: 2
server: envoy

{"authorized":false,"error":"missing Header: authorization"}
```

## start fiware orion on AKS
[fiware orion](https://catalogue-server.fiware.org/enablers/publishsubscribe-context-broker-orion-context-broker)

In [ ]:
kubectl apply -f orion/orion.yaml

In [ ]:
kubectl get pods -l app=orion

example)
```
NAME                     READY     STATUS    RESTARTS   AGE
orion-54f5cdcb5d-d2pt5   1/1       Running   0          56s
orion-54f5cdcb5d-hv274   1/1       Running   0          56s
orion-54f5cdcb5d-xbnx2   1/1       Running   0          56s
```

In [ ]:
kubectl get services -l app=orion

example)
```
NAME      TYPE        CLUSTER-IP    EXTERNAL-IP   PORT(S)    AGE
orion     ClusterIP   10.0.44.126   <none>        1026/TCP   1m
```

In [ ]:
TOKEN=$(cat ${PJ_ROOT}/secrets/auth-tokens.json | jq '.bearer_tokens[0].token' -r)
curl -i -H "Authorization: bearer ${TOKEN}" https://api.${DOMAIN}/orion/v2/entities/

example)
```
HTTP/1.1 200 OK
content-length: 2
content-type: application/json
fiware-correlator: 4731eb48-4dc1-11e8-b1a2-0a580af4010a
date: Wed, 02 May 2018 04:28:35 GMT
x-envoy-upstream-service-time: 5
server: envoy

[]
```

## start duplicate message filter service for idas

In [ ]:
kubectl apply -f idas/fiware-mqtt-msgfilter.yaml

In [ ]:
kubectl get pods -l pod=mqtt-msgfilter

example)
```
NAME                              READY     STATUS    RESTARTS   AGE
mqtt-msgfilter-6f76445596-cmbqz   1/1       Running   0          26s
mqtt-msgfilter-6f76445596-wrhzb   1/1       Running   0          26s
mqtt-msgfilter-6f76445596-znnvg   1/1       Running   0          26s
```

In [ ]:
kubectl get services -l service=mqtt-msgfilter

example)
```
NAME             TYPE        CLUSTER-IP    EXTERNAL-IP   PORT(S)    AGE
mqtt-msgfilter   ClusterIP   10.0.133.42   <none>        5001/TCP   43s
```

## start fiware idas(iotagent-ul) on AKS
[fiware IDAS(iotagent-ul)](https://catalogue-server.fiware.org/enablers/backend-device-management-idas)

**In this demonstration, we use customized iotagent-ul in order to ignore duplicate MQTT messages.**

### build and push the idas conatiner image

In [ ]:
env IOTAGENT_PASSWORD=${VERNEMQ__iotagent} envsubst < idas/iotagent-ul/config.js.template > idas/iotagent-ul/config.js

In [ ]:
docker build -t ${REPOSITORY}/tech-sketch/iotagent-ul:1.6.0 idas/iotagent-ul/

In [ ]:
az acr login --name ${ACR_NAME}

In [ ]:
docker push ${REPOSITORY}/tech-sketch/iotagent-ul:1.6.0

expected)
```
The push refers to repository [ccfdacr.azurecr.io/tech-sketch/iotagent-ul]

0a577855: Preparing 
e42a42ab: Preparing 
e3e1d3a9: Preparing 
a102cc1f: Preparing 
1.6.0: digest: sha256:166a9aad44a8e052412da041a620544341f18b23e86fbc4fa5c9d4b839dc6ba5 size: 1372
```

In [ ]:
az acr repository list --name ${ACR_NAME} --output table

example)
```
Result
---------------------------------
tech-sketch/iotagent-ul
```

### start idas

In [ ]:
envsubst < idas/iotagent-ul.yaml | kubectl apply -f -

In [ ]:
kubectl get pods -l app=iotagent-ul

example)
```
NAME                           READY     STATUS    RESTARTS   AGE
iotagent-ul-79685b64bf-8krps   1/1       Running   0          3m
iotagent-ul-79685b64bf-m6nlg   1/1       Running   0          3m
iotagent-ul-79685b64bf-mjpbl   1/1       Running   0          3m
```

In [ ]:
kubectl get services -l app=iotagent-ul

example)
```
NAME          TYPE        CLUSTER-IP     EXTERNAL-IP   PORT(S)             AGE
iotagent-ul   ClusterIP   10.0.180.155   <none>        4041/TCP,7896/TCP   43s
```

In [ ]:
TOKEN=$(cat secrets/auth-tokens.json | jq '.bearer_tokens[0].token' -r)
curl -i -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${FIWARE_SERVICEPATH}*" https://api.${DOMAIN}/idas/ul20/manage/iot/services/

example)
```
HTTP/1.1 200 OK
x-powered-by: Express
fiware-correlator: c114fc5e-b4a2-40f6-b7fe-1d68369784e5
content-type: application/json; charset=utf-8
content-length: 25
etag: W/"19-WMYe0U6ocKhQjp+oaVnMHLdbylc"
date: Wed, 02 May 2018 06:16:18 GMT
x-envoy-upstream-service-time: 9
server: envoy

{"count":0,"services":[]}
```

## start fiware cygnus on AKS
[fiware cygnus](https://catalogue-server.fiware.org/enablers/cygnus)

**In this demonstration, we use re-configured cygnus in order to revoke unnecessary sinks.**

### build and push the idas conatiner image

In [ ]:
docker build -t ${REPOSITORY}/tech-sketch/cygnus-ngsi:1.8.0 ./cygnus/fiware-cygnus/

In [ ]:
az acr login --name ${ACR_NAME}

In [ ]:
docker push ${REPOSITORY}/tech-sketch/cygnus-ngsi:1.8.0

expected)
```
The push refers to repository [ccfdacr.azurecr.io/tech-sketch/cygnus-ngsi]

c237fee1: Preparing 
422de19a: Preparing 
32a91a75: Preparing 
62ba0f28: Preparing 
1.8.0: digest: sha256:affdaed05cc1762c8dc24e75c1987fdb400320fa69c84dd3138019216fed954a size: 1367
```

In [ ]:
az acr repository list --name ${ACR_NAME} --output table

example)
```
Result
---------------------------------
tech-sketch/cygnus-ngsi
tech-sketch/iotagent-ul
```

### start cygnus

In [ ]:
envsubst < cygnus/cygnus.yaml | kubectl apply -f -

In [ ]:
kubectl get pods -l app=cygnus

example)
```
NAME                      READY     STATUS    RESTARTS   AGE
cygnus-5c68fb6578-fdmtg   1/1       Running   0          44s
cygnus-5c68fb6578-stmds   1/1       Running   0          44s
cygnus-5c68fb6578-z85lp   1/1       Running   0          44s
```

In [ ]:
kubectl get services -l app=cygnus

example)
```
NAME      TYPE        CLUSTER-IP       EXTERNAL-IP   PORT(S)             AGE
cygnus    ClusterIP   10.103.255.240   <none>        5050/TCP,8081/TCP   1m
```

## start command proxy service on AKS

In [ ]:
envsubst < controller/fiware-cmd-proxy.yaml | kubectl apply -f -

In [ ]:
kubectl get pods -l pod=cmd-proxy

example)
```
NAME                        READY     STATUS    RESTARTS   AGE
cmd-proxy-d7cbc6dfb-6hnsp   1/1       Running   0          46s
cmd-proxy-d7cbc6dfb-v6rrx   1/1       Running   0          46s
cmd-proxy-d7cbc6dfb-zf2fd   1/1       Running   0          46s
```

In [ ]:
kubectl get services -l service=cmd-proxy

example)
```
NAME        TYPE        CLUSTER-IP    EXTERNAL-IP   PORT(S)    AGE
cmd-proxy   ClusterIP   10.0.172.56   <none>        8888/TCP   1m
```